<a href="https://colab.research.google.com/github/LikhanBiswas/CNN-Models/blob/main/custom_cnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras import models

In [ ]:
my_model= models.Sequential()

In [ ]:
# Add first convolutional block
my_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(215,215,3)))
my_model.add(MaxPooling2D((2, 2), strides=(2,2), padding='same'))

# second block
my_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2),strides=(2,2), padding='same'))
# third block
my_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), strides=(2,2), padding='same'))
# fourth block
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), strides=(2,2), padding='same'))
#fifth block
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), strides=(2,2), padding='same'))
#sixth block
my_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
my_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
my_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), strides=(2,2), padding='same'))
#seventh block
my_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
my_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
my_model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), strides=(2,2), padding='same'))

In [ ]:
my_model.add(GlobalAveragePooling2D())
# fully connected layer
my_model.add(Dense(64, activation='relu'))
my_model.add(BatchNormalization())
# make predictions
my_model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Show a summary of the model. Check the number of trainable parameters
my_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 215, 215, 16)      448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 108, 108, 16)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 108, 108, 32)      4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 54, 54, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 27, 27, 128)      

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# save best model automatically
mc= ModelCheckpoint('/content/drive/MyDrive/extension/custom_cnn.h5', monitor='val_loss', 
                    mode='min', verbose=1, save_best_only=True)
cb_list=[es,mc]

In [ ]:
x = Flatten()(my_model.output)
prediction = Dense(1, activation='sigmoid')(x)


In [ ]:
my_model.compile(optimizer='adam', loss='binary_crossentropy', 
                 metrics=['accuracy'])

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# set up data generator
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


In [ ]:
train_path = '/content/drive/MyDrive/real vs fake/NEW_TRAIN_10K'
valid_path = '/content/drive/MyDrive/real vs fake/test'

In [ ]:
data_generator = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range=0.1,
                                   rotation_range=90,
                                   width_shift_range=0.1,
                                   #fill_mode='wrap',
                                   height_shift_range=0.1,
                                   #channel_shift_range=150.0,
                                   brightness_range=(0.1, 0.5),
                                   vertical_flip=True,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# get batches of training images from the directory
train_generator = data_generator.flow_from_directory(
        '/content/drive/MyDrive/real vs fake/NEW_TRAIN_10K',
        target_size=(215, 215),
        batch_size=16,
        class_mode='binary')

# get batches of validation images from the directory
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/real vs fake/test',
        target_size=(215, 215),
        batch_size=16,
        class_mode='binary')

Found 10000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [ ]:
history = my_model.fit(
        train_generator,
        epochs=20,
        steps_per_epoch=len(train_generator),
        validation_data=validation_generator,
        validation_steps=667, callbacks=cb_list)

Epoch 1/20
625/625 [==============================] - 1191s 2s/step - loss: 0.0801 - accuracy: 0.9709 - val_loss: 36.9675 - val_accuracy: 0.4992

Epoch 00001: val_loss improved from inf to 36.96752, saving model to /content/drive/MyDrive/extension/custom_cnn.h5
Epoch 2/20
625/625 [==============================] - 163s 262ms/step - loss: 0.0700 - accuracy: 0.9765 - val_loss: 16.6188 - val_accuracy: 0.5025

Epoch 00002: val_loss improved from 36.96752 to 16.61884, saving model to /content/drive/MyDrive/extension/custom_cnn.h5
Epoch 3/20
625/625 [==============================] - 160s 256ms/step - loss: 0.0573 - accuracy: 0.9800 - val_loss: 1.5119 - val_accuracy: 0.5720

Epoch 00003: val_loss improved from 16.61884 to 1.51193, saving model to /content/drive/MyDrive/extension/custom_cnn.h5
Epoch 4/20
625/625 [==============================] - 161s 257ms/step - loss: 0.0541 - accuracy: 0.9811 - val_loss: 14.6001 - val_accuracy: 0.4988

Epoch 00004: val_loss did not improve from 1.51193
Epo